In [1]:
import brainlit
from brainlit.utils.ngl_pipeline import NeuroglancerSession
from brainlit.viz.swc import *
from brainlit.viz.visualize import *
from brainlit.algorithms.generate_fragments import tube_seg, adaptive_thresh
import napari
from skimage import io
import pandas as pd

Using TensorFlow backend.
/home/matt/anaconda3/lib/python3.7/site-packages/napari/__init__.py:38: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.12.1. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


In [2]:
def napari_viewer(img, labels=None, shapes=None, label_name="Segmentation"):
    #with napari.gui_qt():
    viewer = napari.view_image(np.squeeze(np.array(img)))
    if labels is not None:
        viewer.add_labels(labels, name=label_name)
    if shapes is not None:
        viewer.add_shapes(data=shapes, shape_type='path', edge_color='blue', name='Skeleton')
    return viewer

In [3]:
def transform(point, scales, swc_offset, brain_offset, vol_offset):
    im_offset = np.add(brain_offset, vol_offset)
    offset_diff = np.subtract(swc_offset, im_offset)
    return (point + offset_diff)/scales*1000

In [4]:
def create_seed(subneuron_df, scales, swc_offset, brain_offset, vol_offset):
    seed = []
    for index, row in subneuron_df.iterrows():
        point = np.array((row['x'], row['y'], row['z']))
        point = transform(point, scales, swc_offset, brain_offset, vol_offset)
        _, s = adaptive_thresh.get_seed(point)
        seed.append(s)
    print(seed)
    return seed

In [5]:
data_dir = "/home/matt/Documents/brainlit/benchmarking_datasets/"
swc_dir = data_dir + "Manual-GT/"

In [6]:
im_path = data_dir + "validation_1-gfp.tif"
swc_path = swc_dir + "10-01_validation_1-5/10-01_validation_1/tree_1.swc"

In [7]:
scales = np.array([298.66187, 301.37174, 1050.67223])

In [8]:
swc_offset = [73448.511531, 14630.940220, 33145.238458]
brain_offset = [69445.19581378, 12917.40798423, 30199.63896704]
vol_offset = [3944.427317, 1689.489974, 2904.058044]

In [9]:
im = io.imread(im_path).T

In [10]:
%gui qt
napari_viewer(im)

RuntimeError: napari requires a Qt event loop to run. To create one, try one of the following: 
  - use the `napari.gui_qt()` context manager. See https://github.com/napari/napari/tree/master/examples for usage examples.
  - In IPython or a local Jupyter instance, use the `%gui qt` magic command.
  - Launch IPython with the option `--gui=qt`.
  - (recommended) in your IPython configuration file, add or uncomment the line `c.TerminalIPythonApp.gui = 'qt'`. Then, restart IPython.

In [11]:
subneuron_df = pd.read_csv(swc_path, skiprows = 3, header = None, sep = " ")
subneuron_df.columns = ["sample",  "structure",      "x",      "y",      "z",
                         "r",  "parent"]

In [12]:
G = df_to_graph(subneuron_df)
print('Number of nodes:', len(G.nodes))
print('Number of edges:', len(G.edges))
print('\n')
print('Sample 1 coordinates (x,y,z)')
G.nodes[1]

Number of nodes: 127
Number of edges: 126


Sample 1 coordinates (x,y,z)


{'x': 13, 'y': -30, 'z': -13}

In [13]:
paths = graph_to_paths(G)
len(paths)

1

In [14]:
transformed_paths = []
for i in range(len(paths)):
    transformed_paths.append(transform(paths[i], scales, swc_offset, brain_offset, vol_offset))

In [15]:
seed = create_seed(subneuron_df, scales, swc_offset, brain_offset, vol_offset)

[(26, -21, 243), (27, -20, 243), (28, -19, 242), (28, -18, 242), (28, -12, 236), (28, -11, 236), (28, -6, 231), (29, -5, 230), (29, -4, 229), (29, 0, 229), (29, 0, 228), (29, 1, 227), (29, 2, 226), (29, 4, 226), (29, 6, 224), (29, 7, 224), (29, 9, 222), (29, 11, 222), (29, 19, 222), (30, 20, 222), (30, 21, 222), (30, 22, 221), (30, 23, 221), (30, 24, 220), (30, 25, 220), (30, 26, 219), (30, 28, 219), (31, 29, 218), (31, 31, 216), (31, 32, 216), (32, 33, 215), (32, 34, 215), (32, 37, 212), (33, 38, 212), (33, 39, 213), (33, 40, 213), (33, 41, 212), (33, 40, 213), (35, 42, 211), (35, 45, 211), (35, 47, 209), (35, 48, 208), (35, 48, 206), (35, 49, 205), (36, 50, 204), (36, 51, 204), (36, 53, 202), (37, 53, 202), (38, 54, 202), (38, 55, 203), (38, 57, 203), (39, 58, 203), (39, 60, 203), (39, 62, 201), (39, 63, 201), (39, 64, 200), (39, 65, 200), (39, 70, 195), (39, 72, 195), (39, 73, 194), (39, 83, 194), (39, 84, 194), (40, 85, 193), (40, 86, 193), (40, 87, 192), (40, 89, 192), (40, 90, 19

In [16]:
labels = tube_seg.tubes_seg(im, transformed_paths[0], 2, spheres=False)

In [17]:
%gui qt
napari_viewer(im, labels = labels, shapes=transformed_paths[0])

In [53]:
%gui qt
viewer = napari_viewer(im, labels=labels, shapes=transformed_paths, label_name="Tube Segmentation")

In [25]:
## NOTE: Can't have negative coordinates in confidence_connected

In [30]:
labels = adaptive_thresh.confidence_connected_threshold(im, [seed[10], seed[20]], num_iter=1, multiplier=0.1)

In [31]:
print(seed)

[(26, -21, 243), (27, -20, 243), (28, -19, 242), (28, -18, 242), (28, -12, 236), (28, -11, 236), (28, -6, 231), (29, -5, 230), (29, -4, 229), (29, 0, 229), (29, 0, 228), (29, 1, 227), (29, 2, 226), (29, 4, 226), (29, 6, 224), (29, 7, 224), (29, 9, 222), (29, 11, 222), (29, 19, 222), (30, 20, 222), (30, 21, 222), (30, 22, 221), (30, 23, 221), (30, 24, 220), (30, 25, 220), (30, 26, 219), (30, 28, 219), (31, 29, 218), (31, 31, 216), (31, 32, 216), (32, 33, 215), (32, 34, 215), (32, 37, 212), (33, 38, 212), (33, 39, 213), (33, 40, 213), (33, 41, 212), (33, 40, 213), (35, 42, 211), (35, 45, 211), (35, 47, 209), (35, 48, 208), (35, 48, 206), (35, 49, 205), (36, 50, 204), (36, 51, 204), (36, 53, 202), (37, 53, 202), (38, 54, 202), (38, 55, 203), (38, 57, 203), (39, 58, 203), (39, 60, 203), (39, 62, 201), (39, 63, 201), (39, 64, 200), (39, 65, 200), (39, 70, 195), (39, 72, 195), (39, 73, 194), (39, 83, 194), (39, 84, 194), (40, 85, 193), (40, 86, 193), (40, 87, 192), (40, 89, 192), (40, 90, 19

In [32]:
%gui qt
viewer = napari_viewer(im, labels=labels, shapes=transformed_paths, label_name="Region Growing")